In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
import time
import gc  # Garbage collector

In [2]:
# --- Configuration ---
model_name = "meta-llama/Llama-3.1-8B-Instruct"
access_token = ""  # Replace with your token

# Max tokens for the generated *explanation* (adjust as needed)
max_explanation_length = 75
# Path to your saved correct results (adjust if needed)
correct_results_file = '/content/llama3_nli_analysis_500_shap_correct.csv'
# Output file path
output_file = 'results_correct_with_explanations.csv'

# --- Load Model and Tokenizer ---
print("Loading model and tokenizer...")
# Ensure tokenizer is loaded correctly
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
if tokenizer.pad_token is None:
    print("Setting pad_token to eos_token")
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" # Important for Causal LM generation

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=access_token,
    torch_dtype=torch.float16, # Use float16 for efficiency
    device_map="auto" # Automatically distribute across available devices (GPU/CPU)
)
model.eval() # Set model to evaluation mode
print("Model and tokenizer loaded.")

Loading model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting pad_token to eos_token


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model and tokenizer loaded.


In [7]:
# --- Helper Function for Generation ---
def generate_model_explanation(prompt, max_new_tokens):
    """Generates text using the loaded model."""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=tokenizer.model_max_length - max_new_tokens).to(model.device)

    # Ensure attention_mask is passed if padding is used
    attention_mask = inputs.attention_mask

    # Clear GPU cache before generation if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Generate output
    with torch.no_grad():
      outputs = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id, # Make sure EOS token is considered

        # --- CHANGES START HERE ---
        do_sample=True,         # Enable sampling
        temperature=0.1,          # Adjust temperature (e.g., 0.-0.9)
        top_k=50,               # Consider top K tokens
        # top_p=0.9,            # Alternatively, use top_p nucleus sampling
        repetition_penalty=1.15 # Penalize repetition (e.g., 1.1-1.2)
        # --- CHANGES END HERE ---
    )

    # Decode the generated tokens, skipping special tokens and the input prompt
    # We want only the newly generated part
    input_length = inputs.input_ids.shape[1]
    generated_ids = outputs[0, input_length:]
    explanation = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # Basic cleaning
    explanation = explanation.strip()

    # Clear GPU cache after generation
    del inputs, outputs
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect() # Force garbage collection

    return explanation

In [8]:
# --- Load Data ---
print(f"Loading correct results from {correct_results_file}...")
try:
    df_correct = pd.read_csv(correct_results_file)
    # Handle potential loading issues if 'shap_value' was complex
    if 'shap_value' in df_correct.columns:
         # Example: If shap_value needs literal_eval, uncomment below
         # import ast
         # df_correct['shap_value'] = df_correct['shap_value'].apply(ast.literal_eval)
         pass # Add specific handling if needed based on how shap_value was saved
except FileNotFoundError:
    print(f"Error: File not found at {correct_results_file}")
    print("Please ensure the file exists or modify the 'correct_results_file' variable.")
    exit()
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit()

print(f"Loaded {len(df_correct)} correctly classified samples.")


Loading correct results from /content/llama3_nli_analysis_500_shap_correct.csv...
Loaded 179 correctly classified samples.


In [9]:
# --- Generate Explanations ---
model_explanations = []
print(f"Generating model explanations (max_new_tokens={max_explanation_length})...")
start_time = time.time()

# Use tqdm for progress bar
for index, row in tqdm(df_correct.iterrows(), total=df_correct.shape[0], desc="Generating Explanations"):
    premise = row['premise']
    hypothesis = row['hypothesis']
    # Use 'pred_label' as it's confirmed correct for this dataframe
    predicted_label = row['pred_label']

    # Construct the prompt for explanation
    explanation_prompt = f"""Premise: {premise}
                             Hypothesis: {hypothesis}
                             Classification: {predicted_label}
                             Brief Reasoning for this classification (Max 25 words):"""

    # Generate the explanation
    explanation = generate_model_explanation(explanation_prompt, max_new_tokens=max_explanation_length)
    model_explanations.append(explanation)
    print(explanation)

    # Optional: Print progress periodically
    print(f"Processed {index + 1}/{len(df_correct)} samples...")


Generating model explanations (max_new_tokens=75)...


Generating Explanations:   0%|          | 0/179 [00:00<?, ?it/s]

The scene of two women hugging contradicts the violence and aggression implied by "fighting" in front of a deli. This is an example of a contradiction because it shows opposing ideas or actions together. 
This image depicts two women embracing, which suggests warmth, love, and affection. In contrast, the phrase "the men are fighting outside a deli" implies
Processed 1/179 samples...
The premise describes two specific actions of washing hands that directly implies they were wearing numbered jerseys. This is an example of entailment where the conclusion logically follows from the premises.
Processed 2/179 samples...


KeyboardInterrupt: 

In [ ]:
# --- Add Explanations to DataFrame ---
df_correct['baseline_model_explanation'] = model_explanations
print("\nFinished generating explanations.")
end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")


Finished generating explanations.
Total time taken: 281.66 seconds


In [ ]:
# --- Save Updated DataFrame ---
output_file = 'Llama_3_8B_Instruct_explanations.csv'
print(f"\nSaving updated DataFrame to {output_file}...")
try:
    df_correct.to_csv(output_file, index=False)
    print("DataFrame saved successfully.")
except Exception as e:
    print(f"Error saving CSV: {e}")

print("\nScript finished.")


Saving updated DataFrame to results_correct_with_explanations.csv...
DataFrame saved successfully.

Script finished.
